In [ ]:
!mkdir /home/aistudio/external-libraries
!pip install jieba -t /home/aistudio/external-libraries
!pip install pymysql -t /home/aistudio/external-libraries
!pip install wordcloud -t /home/aistudio/external-libraries

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/d6/af/55c7f2aa0997147943b474a74bab8deb17e7cf935b9abb8798d724c57721/wordcloud-1.6.0-cp37-cp37m-manylinux1_x86_64.whl
     |████████████████████████████████| 13.1MB 33kB/s eta 0:00:01�█████▎  | 12.0MB 33kB/s eta 0:00:34
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/89/3e/31c2e5385d7588016c6f7ac552e81c3fff2bef4bc61b6f82f8177752405c/Pillow-6.2.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/9b/af/4fc72f9d38e43b092e91e5b8cb9956d25b2e3ff8c75aed95df5569e4734e/numpy-1.17.4-cp37-cp37m-manylinux1_x86_64.whl
     |████████████████████████████████| 92kB 36.6MB/s eta 0:00:01
     |████████████████████████████████| 235kB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 33.4MB/s eta 0:00:01
     |████████████████████████████████| 583kB 57.0MB/s eta 0:00:01


In [1]:
import sys
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
# -*- coding: utf8 -*-
!pip install paddlehub==1.1.1
!pip install pyahocorasick
import paddlehub as hub

import paddle.fluid as fluid

In [ ]:

module = hub.Module(name="senta_bilstm")
dataset = hub.dataset.ChnSentiCorp()
# 生成Reader
reader = hub.reader.LACClassifyReader(
    dataset=dataset, vocab_path=module.get_vocab_path())
# 选择Fine-Tune优化策略
strategy = hub.AdamWeightDecayStrategy(
    weight_decay=0.01,
    warmup_proportion=0.1,
    learning_rate=5e-5,
    lr_scheduler="linear_decay",
    optimizer_name="adam")
# 选择运行时配置
config = hub.RunConfig(
    use_data_parallel=True,
    use_pyreader=False,
    use_cuda=True,
    num_epoch=1,
    checkpoint_dir="nlp_senta_turtorial_demo",
    batch_size=32,
    log_interval=10,
    eval_interval=50,
    strategy=strategy)
# 组建Finetune Task
inputs, outputs, program = module.context(trainable=True)

sent_feature = outputs["sentence_feature"]

feed_list = [inputs["words"].name]

cls_task = hub.TextClassifierTask(
    data_reader=reader,
    feature=sent_feature,
    feed_list=feed_list,
    num_classes=dataset.num_labels,
    config=config)
# 模型训练
cls_task.finetune_and_eval()

In [ ]:
# 模型预测
#coding:utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import os

import paddle
import paddle.fluid as fluid
import paddlehub as hub

module = hub.Module(name="senta_bilstm")
inputs, outputs, program = module.context(trainable=True)

dataset = hub.dataset.ChnSentiCorp()
reader = hub.reader.LACClassifyReader(
    dataset=dataset, vocab_path=module.get_vocab_path())

strategy = hub.AdamWeightDecayStrategy(
    weight_decay=0.01,
    warmup_proportion=0.1,
    learning_rate=5e-5,
    lr_scheduler="linear_decay",
    optimizer_name="adam")

config = hub.RunConfig(
    use_data_parallel=False,
    use_pyreader=False,
    use_cuda=True,
    batch_size=1,
    enable_memory_optim=False,
    checkpoint_dir="nlp_senta_turtorial_demo",
    strategy=strategy)

sent_feature = outputs["sentence_feature"]

feed_list = [inputs["words"].name]

cls_task = hub.TextClassifierTask(
    data_reader=reader,
    feature=sent_feature,
    feed_list=feed_list,
    num_classes=dataset.num_labels,
    config=config)



[2019-12-06 01:53:56,023] [    INFO] - Installing senta_bilstm module


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/cache/senta_bilstm
[==================================================] 100.00%


[2019-12-06 01:54:17,181] [    INFO] - Successfully installed senta_bilstm-1.0.1


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/cache/lac
[==================================================] 100.00%


[2019-12-06 01:54:20,594] [    INFO] - 13 pretrained paramaters loaded by PaddleHub


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/dataset/chnsenticorp.tar.gz
[==================================================] 100.00%


[2019-12-06 01:54:21,040] [    INFO] - Installing lac module
[2019-12-06 01:54:21,044] [    INFO] - Module lac already installed in /home/aistudio/.paddlehub/modules/lac
[2019-12-06 01:54:23,939] [    INFO] - Checkpoint dir: nlp_senta_turtorial_demo


In [5]:
# 筛选、分词、情感分析、入库
import jieba
import jieba.analyse
import pymysql
import time
import math

TOTAL_DATA=100000
PER_TIME=20
SEARCH_TIME=math.ceil(TOTAL_DATA/PER_TIME)

db = pymysql.connect("localhost","root","123456","weibo" )
cursor = db.cursor()

dataStore=[]
sentimentData=[]

month={'Jan':'1','Feb':'2','Mar':'3','Apr':'4','May':'5','Jun':'6','Jul':'7','Aug':'8','Sep':'9','Oct':'10','Nov':'11','Dec':'12'}
'''
def insertTONew(data):
    insert_sql = """insert into DataFiltered(mblogid,publish_at,content,username,pic_count,gender,followers_count,friends_count,statuses_count,user_created_at,place,created_at,hour,keyword,sentiment)\
         value ('{0[1]}','{0[2]}','{0[3]}','{0[4]}','{0[5]}','{0[6]}','{0[7]}','{0[8]}','{0[9]}','{0[10]}','{0[11]}','{0[12]}','{0[13]}','{0[14]}','{0[15]}'),\
            ('{1[1]}','{1[2]}','{1[3]}','{1[4]}','{1[5]}','{1[6]}','{1[7]}','{1[8]}','{1[9]}','{1[10]}','{1[11]}','{1[12]}','{1[13]}','{1[14]}','{1[15]}'),\
            ('{2[1]}','{2[2]}','{2[3]}','{2[4]}','{2[5]}','{2[6]}','{2[7]}','{2[8]}','{2[9]}','{2[10]}','{2[11]}','{2[12]}','{2[13]}','{2[14]}','{2[15]}'),\
            ('{3[1]}','{3[2]}','{3[3]}','{3[4]}','{3[5]}','{3[6]}','{3[7]}','{3[8]}','{3[9]}','{3[10]}','{3[11]}','{3[12]}','{3[13]}','{3[14]}','{3[15]}'),\
            ('{4[1]}','{4[2]}','{4[3]}','{4[4]}','{4[5]}','{4[6]}','{4[7]}','{4[8]}','{4[9]}','{4[10]}','{4[11]}','{4[12]}','{4[13]}','{4[14]}','{4[15]}'),\
            ('{5[1]}','{5[2]}','{5[3]}','{5[4]}','{5[5]}','{5[6]}','{5[7]}','{5[8]}','{5[9]}','{5[10]}','{5[11]}','{5[12]}','{5[13]}','{5[14]}','{5[15]}'),\
            ('{6[1]}','{6[2]}','{6[3]}','{6[4]}','{6[5]}','{6[6]}','{6[7]}','{6[8]}','{6[9]}','{6[10]}','{6[11]}','{6[12]}','{6[13]}','{6[14]}','{6[15]}'),\
            ('{7[1]}','{7[2]}','{7[3]}','{7[4]}','{7[5]}','{7[6]}','{7[7]}','{7[8]}','{7[9]}','{7[10]}','{7[11]}','{7[12]}','{7[13]}','{7[14]}','{7[15]}'),\
            ('{8[1]}','{8[2]}','{8[3]}','{8[4]}','{8[5]}','{8[6]}','{8[7]}','{8[8]}','{8[9]}','{8[10]}','{8[11]}','{8[12]}','{8[13]}','{8[14]}','{8[15]}'),\
            ('{9[1]}','{9[2]}','{9[3]}','{9[4]}','{9[5]}','{9[6]}','{9[7]}','{9[8]}','{9[9]}','{9[10]}','{9[11]}','{9[12]}','{9[13]}','{9[14]}','{9[15]}'),\
            ('{10[1]}','{10[2]}','{10[3]}','{10[4]}','{10[5]}','{10[6]}','{10[7]}','{10[8]}','{10[9]}','{10[10]}','{10[11]}','{10[12]}','{10[13]}','{10[14]}','{10[15]}'),\
            ('{11[1]}','{11[2]}','{11[3]}','{11[4]}','{11[5]}','{11[6]}','{11[7]}','{11[8]}','{11[9]}','{11[10]}','{11[11]}','{11[12]}','{11[13]}','{11[14]}','{11[15]}'),\
            ('{12[1]}','{12[2]}','{12[3]}','{12[4]}','{12[5]}','{12[6]}','{12[7]}','{12[8]}','{12[9]}','{12[10]}','{12[11]}','{12[12]}','{12[13]}','{12[14]}','{12[15]}'),\
            ('{13[1]}','{13[2]}','{13[3]}','{13[4]}','{13[5]}','{13[6]}','{13[7]}','{13[8]}','{13[9]}','{13[10]}','{13[11]}','{13[12]}','{13[13]}','{13[14]}','{13[15]}'),\
            ('{14[1]}','{14[2]}','{14[3]}','{14[4]}','{14[5]}','{14[6]}','{14[7]}','{14[8]}','{14[9]}','{14[10]}','{14[11]}','{14[12]}','{14[13]}','{14[14]}','{14[15]}'),\
            ('{15[1]}','{15[2]}','{15[3]}','{15[4]}','{15[5]}','{15[6]}','{15[7]}','{15[8]}','{15[9]}','{15[10]}','{15[11]}','{15[12]}','{15[13]}','{15[14]}','{15[15]}'),\
            ('{16[1]}','{16[2]}','{16[3]}','{16[4]}','{16[5]}','{16[6]}','{16[7]}','{16[8]}','{16[9]}','{16[10]}','{16[11]}','{16[12]}','{16[13]}','{16[14]}','{16[15]}'),\
            ('{17[1]}','{17[2]}','{17[3]}','{17[4]}','{17[5]}','{17[6]}','{17[7]}','{17[8]}','{17[9]}','{17[10]}','{17[11]}','{17[12]}','{17[13]}','{17[14]}','{17[15]}'),\
            ('{18[1]}','{18[2]}','{18[3]}','{18[4]}','{18[5]}','{18[6]}','{18[7]}','{18[8]}','{18[9]}','{18[10]}','{18[11]}','{18[12]}','{18[13]}','{18[14]}','{18[15]}'),\
            ('{19[1]}','{19[2]}','{19[3]}','{19[4]}','{19[5]}','{19[6]}','{19[7]}','{19[8]}','{19[9]}','{19[10]}','{19[11]}','{19[12]}','{19[13]}','{19[14]}','{19[15]}')
            """.format(data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],data[10],data[11],data[12],data[13],data[14],data[15],data[16],data[17],data[18],data[19])
    cursor.execute(insert_sql)
    db.commit()
'''
def insertOneToNew(data):
    insert_sql = """insert into DataFiltered(mblogid,publish_at,content,username,pic_count,gender,followers_count,friends_count,statuses_count,user_created_at,place,created_at,hour,keyword,sentiment)\
         value ('{0[1]}','{0[2]}','{0[3]}','{0[4]}','{0[5]}','{0[6]}','{0[7]}','{0[8]}','{0[9]}','{0[10]}','{0[11]}','{0[12]}','{0[13]}','{0[14]}',{0[15]})""".format(data)
    cursor.execute(insert_sql)
    db.commit()

def getTimeStamp(timeString):
        timeformated=timeString[5]+'-'+month[timeString[1]]+'-'+timeString[2]+' '+timeString[3]
        timeArray = time.strptime(timeformated, "%Y-%m-%d %H:%M:%S")
        timeStamp = int(time.mktime(timeArray))
        return timeStamp
        
def domain(se_sql):
    global dataStore
    global sentimentData
    global PER_TIME
    f=open('./work/test.txt','a')
    
    try:
        cursor.execute(se_sql)
        results=cursor.fetchall()
        for row in results:
            date=row[2].split(' ')
            timehour=date[3].split(':')
            if int(timehour[0])<6:
                timeSp=getTimeStamp(row[2].split(' '))
                insertData=list(row)
                insertData[2]=timeSp
                insertData[3]=insertData[3].replace("\n", " ").replace("\""," ")
                keyword=' '.join(jieba.analyse.extract_tags(insertData[3]))  #分词
                f.write(keyword+'\n')
                insertData.append(timehour[0])
                insertData.append(keyword)
                
                run_states = cls_task.predict(data=[insertData[3]])   #模型预测
                results = [run_state.run_results for run_state in run_states]
                # TODO 修改
                if len(results)>0:
                    res=res=np.argmax(results[0], axis=2)[0]
                    if len(res)>0:
                        insertData.append(res[0])
                    else:
                        insertData.append(3)
                else:
                    insertData.append(4)
                # dataStore.append(insertData)
                # sentimentData.append(insertData[3])
                insertOneToNew(insertData)
                # insertTONew(insertData)
            # if len(dataStore)==PER_TIME:
            #     print("daol-------"+str(len(sentimentData)))
            #     run_states = cls_task.predict(data=sentimentData)
            #     results = [run_state.run_results for run_state in run_states]
            #     for index,OneData in enumerate(dataStore):
            #         res=np.argmax(results[index], axis=2)[0]
            #         OneData.append(res[0])
            #     insertTONew(dataStore)
            #     sentimentData=[]
            #     dataStore=[]        
    except Exception as e:
        print("err happen"+str(e))
    f.close()

#每次取{PER_TIME}条数据    
i=0
bid=0
while i<SEARCH_TIME:
    if i==0:
        sql="SELECT id from DataUnion limit 1"
        cursor.execute(sql)
        res=cursor.fetchone()
        bid=res[0]
    else:
        bid=PER_TIME+bid
    if i%10==0:
        print("------正在进行第%d轮,起始ID为%d------"%(i,bid))
    s_sql="SELECT * FROM DataUnion where id>=%d limit %d"%(bid,PER_TIME)
    domain(s_sql)
    i+=1

# for oneData in dataStore:
#     insertOneToNew(oneData)

db.close()


In [13]:
# 绘制词云
from wordcloud import WordCloud,STOPWORDS
from imageio import imread


mk=imread("./work/background.jpg")
f=open(u'./work/test.txt','r').read()

stopwords = set(STOPWORDS)
word=WordCloud(font_path='./work/kaitiGB2312.ttf',mask=mk,background_color='white',scale=16,collocations=False, width=1280,height=960,margin=2,stopwords=stopwords).generate(f)

word.to_file('./work/word2.png')